## This is a notebook that verify the communication of the pointing kernel with M2 (the rotator is also used)

In this notebook, we 
* point the telescope to a particular pointing, 
* make sure the mt telemetry is correct
* make sure that the zenith angle is picked up by M2 via the subscription
* check that M2 LUT forces vary with the zenith angle acccordingly

When we run this on the summit, we do not use the pointing component. We simply use a mount Controller to publish the mount elevation angle.

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
#iers.conf.auto_download = True

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from lsst.ts.idl.enums import MTM2

plt.jet();

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

usePtg = 1 #if using ptg+mount for telemetry change
#usePtg = 0 

In [ ]:
if summit:
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7) 

In [ ]:
if usePtg:
    mtcs = MTCS(script.domain)
    mtcs.set_rem_loglevel(40)

In [ ]:
if usePtg:
    await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
m2 = mtcs.rem.mtm2
ptg = mtcs.rem.mtptg
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator

In [ ]:
await ptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sstate = await m2.evt_summaryState.aget(timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
if summit:
    if sstate.summaryState == salobj.State.OFFLINE:
        dstate = await m2.evt_detailedState.aget(timeout=5)
        print('m2 state', dstate.detailedState, pd.to_datetime(dstate.private_sndStamp, unit='s'))
if usePtg:
    state = await ptg.evt_summaryState.aget(timeout=5)
    print('ptg', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
    state = await rot.evt_summaryState.aget(timeout=5)
    print('rot', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
    state = await mount.evt_summaryState.aget(timeout=5)
    print('mount', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))

In [ ]:
#execute once to get the state UP one level
m2.evt_summaryState.flush()
if sstate.summaryState == salobj.State.FAULT:
    await m2.cmd_clearError.set_start()
elif sstate.summaryState == salobj.State.OFFLINE:
    await m2.cmd_enterControl.set_start()
elif sstate.summaryState == salobj.State.STANDBY:#system starts applying LUT forces once in STANDBY
    await m2.cmd_start.set_start()
elif sstate.summaryState == salobj.State.DISABLED:#system starts applying FB forces once in DISABLED
    await m2.cmd_enable.set_start()
sstate = await m2.evt_summaryState.next(flush=False, timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
#execute once to get the state OUT one level
if sstate.summaryState == salobj.State.ENABLED:
    await m2.cmd_disable.set_start()
elif sstate.summaryState == salobj.State.DISABLED:
    await m2.cmd_standby.set_start()
elif sstate.summaryState == salobj.State.STANDBY:
    #the below would shut down the CSC; be ready to restart it if you want to try.
    await m2.cmd_exitControl.set_start()
sstate = await m2.evt_summaryState.next(flush=True, timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

In [ ]:
#should work, but doesn't so far, unless we are already in disabled state. A bug that Tiago will talk to TW about.
await salobj.set_summary_state(m2, salobj.State.ENABLED) 

Check the state of the system

In [ ]:
axialForces = await m2.tel_axialForce.aget(timeout=2)
tangentForces = await m2.tel_tangentForce.aget(timeout=2)

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))

In [ ]:
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

In [ ]:
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)

### Do a slew (only for NCSA)

In [ ]:
if summit and usePtg:
    #comment and uncomment the below depending on which components are expected to be on
    #by default, MTCS checks on every component (during the slew).
    mtcs.check.mtaos = False
    mtcs.check.mtm1m3 = False
    mtcs.check.mtm2 = False
    mtcs.check.mthexapod_1 = False
    mtcs.check.mthexapod_2 = False
    mtcs.check.mtdome = False
    mtcs.check.mtdometrajectory = False

In [ ]:
if usePtg:
    alt = 84 * u.deg
    az = -0 * u.deg

    target_name="TMA motion test"
    time_data = await ptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_ptg = Time(time_data.mjd, format="mjd", scale="tai")
    time_err = curr_time_ptg - Time.now()
    print(f"Time error={time_err.sec:0.2f} sec")

    print(curr_time_ptg.tai.value)

    cmd_elaz = AltAz(alt=alt, az=az, 
                    obstime=curr_time_ptg.tai, 
                    location=mtcs.location)
    cmd_radec = cmd_elaz.transform_to(ICRS)
    await mtcs.slew_icrs(ra=cmd_radec.ra, dec=cmd_radec.dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
#if on the summit, 
#   watch chronograf to make sure the elevation angle is well above 82 deg. (We know M2 will fault below that.)
#.  ask Te-Wei to switch on M2 with mount telemetry as source of elevation angle, if we haven't done so.

In [ ]:
#the below gets rejected, with an "elevation out of range" error. 
#even though it is our understanding that this target will never be higher than 86 deg.
#. and the limit has been set at 86.5 deg.
#With newer versions of MTCS, the error will tell us what is the elevation calculated, vs the range limit
# With the example below, lowering the dec to -34.5 works.
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
ra = time_and_date.lst -0.2
dec = -34
print(ra)
await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
dec = -34.
for j in range(2):
    time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
    ra = time_and_date.lst + 0.5
    for i in range(2):
        print(ra, dec)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)
        await asyncio.sleep(39.)
        ra -= 3.5/15.0
if usePtg:
    await ptg.cmd_stopTracking.set_start(timeout=5.)

In [ ]:
mtcs.slew_icrs?

In [ ]:
if usePtg:
    mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
    rotStatus = await rot.evt_inPosition.aget(timeout=5.)
    print('Are we tracking?', mountStatus.elevation , mountStatus.azimuth , rotStatus.inPosition)

In [ ]:
await mtcs.stop_tracking()

### Check telemetry: list here what we want to check once the slew is done

In [ ]:
# Check that the mirror is in Position --  this check is a little more tricky if we are in closed loop. 
m2InPosition = await m2.evt_m2AssemblyInPosition.aget(timeout=10.)
print("Is the m2 in Position after a slew?", m2InPosition.inPosition)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)

### Alternatively, we could create a Controller object to send mount telemetry (for NCSA OR summit)

If we are to use the Controller object, we need to 

* take the mount out of the disabled/enabled states so that it stops sending telemetry data. (check the mount telemetry, via EFD or here in the notebook)
* Get fake telemetry ready (using mountTelGenerator.ipynb) - dAngle is figured out from m2_diagnostics_EFD.ipynb
* Then we need to point m2 to the fake mount elevation

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)

In [ ]:
m2AngleSource = await m2.evt_inclinationTelemetrySource.aget(timeout=10.)
print("Inclinometer Source", MTM2.InclinationTelemetrySource(m2AngleSource.source), 
     pd.to_datetime(m2AngleSource.private_sndStamp, unit='s'))

In [ ]:
if not summit:
    await m2.cmd_selectInclinationSource.set_start(source = MTM2.InclinationTelemetrySource.MTMOUNT)
#await m2.cmd_selectInclinationSource.set_start(source = MTM2.InclinationTelemetrySource.ONBOARD)

In [ ]:
m2AngleSource = await m2.evt_inclinationTelemetrySource.aget(timeout=10.)
print("Inclinometer Source", MTM2.InclinationTelemetrySource(m2AngleSource.source))

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

In [ ]:
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)

In [ ]:
axialForces = await m2.tel_axialForce.aget(timeout=2)
print(pd.to_datetime(axialForces.private_sndStamp, unit='s'))
tangentForces = await m2.tel_tangentForce.aget(timeout=2)

In [ ]:
plt.plot(axialForces.measured)

In [ ]:
plt.plot(tangentForces.measured)

In [ ]:
plt.plot(axialForces.hardpointCorrection)

In [ ]:
plt.plot(tangentForces.hardpointCorrection)

In [ ]:
axialSteps = await m2.tel_axialActuatorSteps.aget(timeout=2)
print(pd.to_datetime(axialSteps.private_sndStamp, unit='s'))
tangentSteps = await m2.tel_tangentActuatorSteps.aget(timeout=2)
axialEncoder = await m2.tel_axialEncoderPositions.aget(timeout=2)
tangentEncoder = await m2.tel_tangentEncoderPositions.aget(timeout=2)

In [ ]:
plt.plot(axialEncoder.position)

In [ ]:
plt.plot(tangentEncoder.position)

### try a different dAngle:

## Now 

set it using mountTelGenerator.ipynb

In [ ]:
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.actualPosition)

#this gives enough time for us to collect later to be analyzed later
await asyncio.sleep(15)

In [ ]:
errorCode = await m2.evt_errorCode.aget(timeout=10.)
print(errorCode.errorCode, pd.to_datetime(errorCode.private_sndStamp, unit='s'))
print(errorCode.errorReport, pd.to_datetime(errorCode.private_sndStamp, unit='s'))

In [ ]:
sstate = await m2.evt_summaryState.aget(timeout=5)
print('starting with: m2 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))

## Stop the system: stop tracking, bring everything else back to enabled (NCSA), or OFFLINE (summit hardware) and STANDBY (summit simulator).

In [ ]:
await ptg.cmd_stopTracking.start(timeout=10.)

In [ ]:
#the rotator was in fault state
await rot.cmd_clearError.set_start()
#await salobj.set_summary_state(rot, salobj.State.DISABLED)
# you may not be able to enable the rotator (it goes to fault) if mount is not publishing CCW telemetry
await salobj.set_summary_state(rot, salobj.State.ENABLED) #,settingsToApply="default") #enable 

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)